<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project3




|Feature|Type|Dataset|Description|
|---|---|---|---|
|title_book|object|df_books|The column contains the title of books.|
|title_series_book|object|df_books|The column contains the title of books series.|
|author_name|object|df_books|The author's name for the books.|
|has_series|object|df_books|The column shows if the book has a series or not.|
|series_number|int32|df_books|The number of series books.|
|avg_rating|float64|df_books|The number of votes for each rating (1 star, 2 stars, 3 stars, 4 stars, 5 stars) and gives you the mean rating.|
|total_rating|int32|df_books|The number of all votes for each rating.|
|voters|int32|df_books|The number of voters who add the book in their books list's on the website.|
|score|int32|df_books|The book's score based on multiple factors, including the number of voters who have voted for it and how highly those voters ranked the book in their books list's on the website, which they can rank the books in order of 1 to 100  on their list.|



In [23]:
import requests
import re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from random import randint
from html.parser import HTMLParser

In [24]:
# make a list for url
url_links=[]

In [25]:
# Used for loop to append all the url pages
for i in range(0,13):
    
    url_links.append('https://www.goodreads.com/list/show/11042.Favourite_young_adult_books' + '?page=' + str(i))

In [26]:
# created a list for all informations that i found it in url
book_n=[]
author_n=[]
rating_n=[]
score_n=[]

In [27]:
# used for loop to create a soup object with the html
for i in range(0,13):
    
    url=url_links[i]
    request = requests.get(url)
    html = request.text # getting the string in title
    soup1 = BeautifulSoup(html,'html.parser')
    item_list1= soup1.find_all('tr') 
# used for loop to find the class form the html and append it to the list..
    for j in item_list1:
        book_name= j.find('a',class_="bookTitle").text
        author_name= j.find('a', class_="authorName").text
        rating= j.find('span', class_="minirating").text
        score= j.find('span', class_="smallText uitext").text
        book_n.append(book_name)
        author_n.append(author_name)
        rating_n.append(rating)
        score_n.append(score)
   
    

In [48]:
# make a dataframe for the lists
df_books=pd.DataFrame({
    'book_n':book_n,
    'author_name':author_n,
    'ratings':rating_n,
    'n_score':score_n,
})
df_books
    

,book_n,author_name,ratings,n_score
0,"\nThe Hunger Games (The Hunger Games, #1)\n",Suzanne Collins,"4.33 avg rating — 6,086,305 ratings","\nscore: 19,286,\n and\n196 peopl..."
1,"\nCatching Fire (The Hunger Games, #2)\n",Suzanne Collins,"4.29 avg rating — 2,360,261 ratings","\nscore: 13,376,\n and\n138 peopl..."
2,"\nMockingjay (The Hunger Games, #3)\n",Suzanne Collins,"4.04 avg rating — 2,217,517 ratings","\nscore: 9,442,\n and\n99 people ..."
3,"\nDivergent (Divergent, #1)\n",Veronica Roth,"4.20 avg rating — 2,767,407 ratings","\nscore: 9,119,\n and\n97 people ..."
4,"\nVampire Academy (Vampire Academy, #1)\n",Richelle Mead,"4.13 avg rating — 527,029 ratings","\nscore: 8,292,\n and\n86 people ..."
...,...,...,...,...
1295,\nLament: The Faerie Queen's Deception (Books ...,Maggie Stiefvater,"3.67 avg rating — 31,207 ratings","\nscore: 58,\n and\n2 people vote..."
1296,\nParrotfish\n,Ellen Wittlinger,"3.71 avg rating — 5,671 ratings","\nscore: 58,\n and\n1 person vote..."
1297,\nTravels with Gannon and Wyatt: Great Bear Ra...,Patti Wheeler,4.19 avg rating — 67 ratings,"\nscore: 58,\n and\n1 person vote..."
1298,"\nUnited We Spy (Gallagher Girls, #6)\n",Ally Carter,"4.44 avg rating — 29,093 ratings","\nscore: 58,\n and\n1 person vote..."


In [49]:
# len(df_books)

### Extract more features from the columns 

In [50]:
# used Regular Expressions to extract all the features i need in the dataframes
df_books['title_book'] = df_books.book_n.str.extract('\n(\D.+)\n', expand=False)
df_books['title_series_book'] = df_books.book_n.str.extract('(\(.*?\))', expand=False)
df_books['series_number'] = df_books.book_n.str.extract('([0-9])', expand=False)
df_books['avg_rating'] = df_books.ratings.str.extract('(\d.+)avg', expand=False)
df_books['total_rating'] = df_books.ratings.str.extract('(\d,.+)ratings', expand=False)
df_books['score'] = df_books.n_score.str.extract('(\d,.+),\n', expand=False)
df_books['voters'] = df_books.n_score.str.extract('\n(\d+)', expand=False)

df_books.head()

,book_n,author_name,ratings,n_score,title_book,title_series_book,series_number,avg_rating,total_rating,score,voters
0,"\nThe Hunger Games (The Hunger Games, #1)\n",Suzanne Collins,"4.33 avg rating — 6,086,305 ratings","\nscore: 19,286,\n and\n196 peopl...","The Hunger Games (The Hunger Games, #1)","(The Hunger Games, #1)",1,4.33,"6,086,305","9,286",196
1,"\nCatching Fire (The Hunger Games, #2)\n",Suzanne Collins,"4.29 avg rating — 2,360,261 ratings","\nscore: 13,376,\n and\n138 peopl...","Catching Fire (The Hunger Games, #2)","(The Hunger Games, #2)",2,4.29,"2,360,261","3,376",138
2,"\nMockingjay (The Hunger Games, #3)\n",Suzanne Collins,"4.04 avg rating — 2,217,517 ratings","\nscore: 9,442,\n and\n99 people ...","Mockingjay (The Hunger Games, #3)","(The Hunger Games, #3)",3,4.04,"2,217,517","9,442",99
3,"\nDivergent (Divergent, #1)\n",Veronica Roth,"4.20 avg rating — 2,767,407 ratings","\nscore: 9,119,\n and\n97 people ...","Divergent (Divergent, #1)","(Divergent, #1)",1,4.20,"2,767,407","9,119",97
4,"\nVampire Academy (Vampire Academy, #1)\n",Richelle Mead,"4.13 avg rating — 527,029 ratings","\nscore: 8,292,\n and\n86 people ...","Vampire Academy (Vampire Academy, #1)","(Vampire Academy, #1)",1,4.13,"7,029","8,292",86


In [51]:
# replace some characters from the features
df_books['title_series_book']= df_books['title_series_book'].str.replace('#','')
df_books['title_series_book']= df_books['title_series_book'].str.replace('(\s\d+)','')
df_books['title_series_book']= df_books['title_series_book'].str.replace('(','').str.replace(')','')
df_books['title_series_book']= df_books['title_series_book'].str.replace(',','')
df_books['title_book'] = df_books['title_book'].str.replace('(\(.*?\))','')
df_books.score = df_books.score.str.replace(',', '')
df_books.total_rating = df_books.total_rating.str.replace(',', '')
df_books.head()

,book_n,author_name,ratings,n_score,title_book,title_series_book,series_number,avg_rating,total_rating,score,voters
0,"\nThe Hunger Games (The Hunger Games, #1)\n",Suzanne Collins,"4.33 avg rating — 6,086,305 ratings","\nscore: 19,286,\n and\n196 peopl...",The Hunger Games,The Hunger Games,1,4.33,6086305,9286,196
1,"\nCatching Fire (The Hunger Games, #2)\n",Suzanne Collins,"4.29 avg rating — 2,360,261 ratings","\nscore: 13,376,\n and\n138 peopl...",Catching Fire,The Hunger Games,2,4.29,2360261,3376,138
2,"\nMockingjay (The Hunger Games, #3)\n",Suzanne Collins,"4.04 avg rating — 2,217,517 ratings","\nscore: 9,442,\n and\n99 people ...",Mockingjay,The Hunger Games,3,4.04,2217517,9442,99
3,"\nDivergent (Divergent, #1)\n",Veronica Roth,"4.20 avg rating — 2,767,407 ratings","\nscore: 9,119,\n and\n97 people ...",Divergent,Divergent,1,4.20,2767407,9119,97
4,"\nVampire Academy (Vampire Academy, #1)\n",Richelle Mead,"4.13 avg rating — 527,029 ratings","\nscore: 8,292,\n and\n86 people ...",Vampire Academy,Vampire Academy,1,4.13,7029,8292,86


In [52]:
# Removed some features that i don't need it anymore..
df_books.drop(['book_n','ratings','n_score'], axis = 1, inplace=True)

### Cleaning the data

In [53]:
# used .info() to checked the data 
df_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1300 entries, 0 to 1299
Data columns (total 8 columns):
author_name          1300 non-null object
title_book           1296 non-null object
title_series_book    812 non-null object
series_number        824 non-null object
avg_rating           1300 non-null object
total_rating         786 non-null object
score                170 non-null object
voters               1300 non-null object
dtypes: object(8)
memory usage: 81.4+ KB


In [54]:
# Replace NaNs with 0 for the numeric features and NA for string features..
df_books['score'].fillna(0, inplace=True)
df_books['total_rating'].fillna(0, inplace=True)
df_books['series_number'].fillna(0, inplace=True)
df_books['title_book'].fillna('NA', inplace=True)
df_books['title_series_book'].fillna('NA', inplace=True)

In [55]:
# used apply to change the type of avg_rating feature to Float
df_books["avg_rating"] = df_books["avg_rating"].apply(pd.to_numeric)

In [56]:
# used .astype to change the type of some features to intger
df_books['score']= df_books['score'].astype(int)
df_books['total_rating']= df_books['total_rating'].astype(int)
df_books['voters']= df_books['voters'].astype(int)
df_books['series_number']= df_books['series_number'].astype(int)

In [57]:
df_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1300 entries, 0 to 1299
Data columns (total 8 columns):
author_name          1300 non-null object
title_book           1300 non-null object
title_series_book    1300 non-null object
series_number        1300 non-null int32
avg_rating           1300 non-null float64
total_rating         1300 non-null int32
score                1300 non-null int32
voters               1300 non-null int32
dtypes: float64(1), int32(4), object(3)
memory usage: 61.1+ KB


In [58]:
# add new feature depaned on the series_number feature
# i have to change the type of the series_number feature to int first to create the new feature..
# make a list
series= [] 
  
# For each row in the column 
for s in df_books['series_number']:        
    if s >= 1:                   # if Voter series 
        series.append('True') 
    else: 
        series.append('False') 
  
 # Create a feature from the list 
df_books['has_series'] = series 
df_books.head()

,author_name,title_book,title_series_book,series_number,avg_rating,total_rating,score,voters,has_series
0,Suzanne Collins,The Hunger Games,The Hunger Games,1,4.33,6086305,9286,196,True
1,Suzanne Collins,Catching Fire,The Hunger Games,2,4.29,2360261,3376,138,True
2,Suzanne Collins,Mockingjay,The Hunger Games,3,4.04,2217517,9442,99,True
3,Veronica Roth,Divergent,Divergent,1,4.20,2767407,9119,97,True
4,Richelle Mead,Vampire Academy,Vampire Academy,1,4.13,7029,8292,86,True


In [59]:
# reindex the dataframe..
column_names = ["title_book","title_series_book","author_name","has_series","series_number","avg_rating","total_rating","voters","score"]
df_books = df_books.reindex(columns=column_names)
df_books.head()

,title_book,title_series_book,author_name,has_series,series_number,avg_rating,total_rating,voters,score
0,The Hunger Games,The Hunger Games,Suzanne Collins,True,1,4.33,6086305,196,9286
1,Catching Fire,The Hunger Games,Suzanne Collins,True,2,4.29,2360261,138,3376
2,Mockingjay,The Hunger Games,Suzanne Collins,True,3,4.04,2217517,99,9442
3,Divergent,Divergent,Veronica Roth,True,1,4.20,2767407,97,9119
4,Vampire Academy,Vampire Academy,Richelle Mead,True,1,4.13,7029,86,8292


In [60]:
# Saved the df_books into csv file.. 
df_books.to_csv('df_books.csv')